1. Инсталируем pyspark
2. Импортируем библиотеки
3. Cоздаем точку входа










In [84]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [85]:
spark = SparkSession.builder \
        .appName("ProductCategoryPairs") \
        .getOrCreate()

Создаем 3 датафрейма: продукты, категории и их связи

>*products_df*

>*categories_data*

> *associations_data*
# Связи:
1. Kiwi - Stone Fruits
2. Pomegranate - Stone Fruits
3. Cabbage - Leafy Vegetables, Cruciferous
4. Zucchini - Leafy Vegetables
5. Raspberry - Berries, Fruits
6. Blackberry - Nightshades, Berries
7. Eggplant - Nightshades, Berries
8. Spinach - Leafy Vegetables
9. Radish - Nightshades, Root Vegetables
10. Mango - Fruits, Stone Fruits, Citrus

In [77]:
products_df = spark.createDataFrame([
    (1, 'Kiwi'),
    (2, 'Pomegranate'),
    (3, 'Cabbage'),
    (4, 'Zucchini'),
    (5, 'Raspberry'),
    (6, 'Blackberry'),
    (7, 'Eggplant'),
    (8, 'Spinach'),
    (9, 'Radish'),
    (10, 'Mango')],
    ['id', 'product_name', ])

In [78]:
categories_df = spark.createDataFrame([
    (1, 'Fruits'),
    (2, 'Stone Fruits'),
    (3, 'Citrus'),
    (4, 'Legumes'),
    (5, 'Peppers'),
    (6, 'Nightshades'),
    (7, 'Cruciferous'),
    (8, 'Root Vegetables'),
    (9, 'Tropical'),
    (10, 'Peppers'), ],
    ['id', 'category_name', ]
)

In [79]:
associations_df = spark.createDataFrame([
    (1, [2]),
    (2, [2]),
    (3, [7]),
    (4, []),
    (5, [5, 1]),
    (6, [6, 5]),
    (7, [6, 5]),
    (8, []),
    (9, [6, 8]),
    (10, [1, 2, 3]), ],
    ['id_product', 'id_categories', ]
)

In [80]:
print("Продукты:")
products_df.show()
print("Категории:")
categories_df.show()
print("Связи:")
associations_df.show()

Продукты:
+---+------------+
| id|product_name|
+---+------------+
|  1|        Kiwi|
|  2| Pomegranate|
|  3|     Cabbage|
|  4|    Zucchini|
|  5|   Raspberry|
|  6|  Blackberry|
|  7|    Eggplant|
|  8|     Spinach|
|  9|      Radish|
| 10|       Mango|
+---+------------+

Категории:
+---+---------------+
| id|  category_name|
+---+---------------+
|  1|         Fruits|
|  2|   Stone Fruits|
|  3|         Citrus|
|  4|        Legumes|
|  5|        Peppers|
|  6|    Nightshades|
|  7|    Cruciferous|
|  8|Root Vegetables|
|  9|       Tropical|
| 10|        Peppers|
+---+---------------+

Связи:
+----------+-------------+
|id_product|id_categories|
+----------+-------------+
|         1|          [2]|
|         2|          [2]|
|         3|          [7]|
|         4|           []|
|         5|       [5, 1]|
|         6|       [6, 5]|
|         7|       [6, 5]|
|         8|           []|
|         9|       [6, 8]|
|        10|    [1, 2, 3]|
+----------+-------------+



+ Разворачиваем ассоциации в отдельные строки
+ Объединяем с продуктами и категориями без продуктов, у которых нет категорий
+ Получаем имена всех продуктов, у которых нет категорий

In [ ]:
exploded_associations_df = associations_df \
    .withColumn("id_category", F.explode("id_categories")) \
    .select("id_product", "id_category")

In [ ]:
result_df = exploded_associations_df \
    .join(products_df, exploded_associations_df.id_product == products_df.id, "inner") \
    .join(categories_df, exploded_associations_df.id_category == categories_df.id, "inner") \
    .select(products_df.product_name, categories_df.category_name)

In [ ]:
no_category_df = products_df.join(exploded_associations_df, products_df.id == exploded_associations_df.id_product, "left_anti")

Получаем все пары **«Имя продукта – Имя категории»** и **Имена всех продуктов, у которых нет категорий**

In [ ]:
print("Все пары 'Имя продукта - Имя категории':")
result_df.show()


Все пары 'Имя продукта - Имя категории':
+------------+---------------+
|product_name|  category_name|
+------------+---------------+
|    Eggplant|        Peppers|
|  Blackberry|        Peppers|
|   Raspberry|        Peppers|
|       Mango|         Fruits|
|   Raspberry|         Fruits|
|       Mango|         Citrus|
|       Mango|   Stone Fruits|
| Pomegranate|   Stone Fruits|
|        Kiwi|   Stone Fruits|
|     Cabbage|    Cruciferous|
|      Radish|    Nightshades|
|    Eggplant|    Nightshades|
|  Blackberry|    Nightshades|
|      Radish|Root Vegetables|
+------------+---------------+



In [ ]:
print("Продукты без категорий:")
no_category_df.select("product_name").show()

Продукты без категорий:
+------------+
|product_name|
+------------+
|    Zucchini|
|     Spinach|
+------------+



In [83]:
spark.stop()